A basic momentum strategy

In [1]:
import pandas as pd

from quant.data import get_returns
from quant.metric import sharpe_ratio

# Parameters of our strategy
univ = ['BTCUSDT','ETHUSDT','ADAUSDT','BNBUSDT','XRPUSDT','DOTUSDT','MATICUSDT']
data_period = 4 # in hours
train_start = '2020-01-01'
train_end = '2022-12-31'
test_start = '2023-01-01'
test_end = '2023-12-31'

freq = f'{data_period}h'
train_returns = get_returns(univ, freq, start_ts=train_start, end_ts=train_end)
train_returns.head(3)

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2019-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-12-31 23:00:00,-0.003487,0.002612,0.003628,-0.003699,0.005771,NaN,NaN
2020-01-01 03:00:00,-0.001346,0.002452,0.000301,-0.003531,-0.008198,NaN,NaN


In [2]:
from quant.algorithms.classic.momentum import rolling_avg_rank

# For each of the following multiples of data_period, we make a momentum strategy and calculate the strategy return:
horizons = [1, 2, 3, 4, 5, 6]
strats_returns = {}
for h in horizons:
    # Calculate the portfolio weights for the strategy
    weights = rolling_avg_rank(train_returns, h)
    # Calculate the return of the strategy for each period
    strats_returns[h] = (weights.shift()*train_returns).sum(axis=1)
strats_returns = pd.DataFrame(strats_returns)
strats_returns.head(5)

,1,2,3,4,5,6
2019-12-31 19:00:00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2019-12-31 23:00:00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
2020-01-01 03:00:00,-1.281039e-03,-0.001281,-0.001281,-0.001281,-0.001281,-0.001281
2020-01-01 07:00:00,2.181741e-03,0.001558,0.001558,0.001558,0.001558,0.001558
2020-01-01 11:00:00,-7.816796e-07,0.000616,0.002451,0.002451,0.002451,0.002451


In [3]:
from quant.metric import sharpe_ratio

# Calculate the annualized Sharpe of each strategy
sr = sharpe_ratio(strats_returns, data_period)
sr

1   -0.989473
2   -0.238533
3    0.259369
4    1.071280
5    1.490998
6    0.899934
dtype: float64

In [4]:
test_returns = get_returns(univ, freq, start_ts=test_start, end_ts=test_end)
test_returns.head(3)

,BTCUSDT,ETHUSDT,ADAUSDT,BNBUSDT,XRPUSDT,DOTUSDT,MATICUSDT
2022-12-31 19:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31 23:00:00,0.000410,0.000946,-0.000326,-0.004892,NaN,0.004662,-0.001328
2023-01-01 03:00:00,0.001353,0.000711,0.002937,0.001229,NaN,0.004640,0.003989


In [5]:
from quant.metric import strategy_returns

# Since 5*4=20 hours gained the highest sharpe ratio, we pick this value as the trained parameter of our strategy
horizon_trained = 20

# Apply the trained strategy to the test data
weights = rolling_avg_rank(test_returns, horizon_trained)
# Calculate the return of the strategy over each test period
strat_returns = strategy_returns(weights, test_returns)
strat_returns

2022-12-31 19:00:00    0.000000
2022-12-31 23:00:00    0.000000
2023-01-01 03:00:00    0.000313
2023-01-01 07:00:00    0.000131
2023-01-01 11:00:00    0.003304
                         ...   
2023-12-30 03:00:00    0.003509
2023-12-30 07:00:00   -0.003829
2023-12-30 11:00:00   -0.001690
2023-12-30 15:00:00    0.004224
2023-12-30 19:00:00    0.005529
Freq: 4h, Length: 2185, dtype: float64

In [6]:
sharpe = sharpe_ratio(strat_returns, data_period)
sharpe

np.float64(0.756299226388108)

In [7]:
strat_returns_cumulative = strat_returns.cumsum()
print(strat_returns_cumulative)

2022-12-31 19:00:00    0.000000
2022-12-31 23:00:00    0.000000
2023-01-01 03:00:00    0.000313
2023-01-01 07:00:00    0.000445
2023-01-01 11:00:00    0.003748
                         ...   
2023-12-30 03:00:00    0.114554
2023-12-30 07:00:00    0.110725
2023-12-30 11:00:00    0.109035
2023-12-30 15:00:00    0.113259
2023-12-30 19:00:00    0.118788
Freq: 4h, Length: 2185, dtype: float64


In [8]:
import importlib
import quant.metric
importlib.reload(quant.metric)

from quant.metric import max_drawdown_stats

stats = max_drawdown_stats(strat_returns_cumulative)
stats

{'value': np.float64(-2.390973303713002),
 'peak_date': Timestamp('2023-01-08 19:00:00'),
 'trough_date': Timestamp('2023-11-30 15:00:00'),
 'recovery_date': Timestamp('2023-12-13 15:00:00'),
 'running_drawdown': 2022-12-31 19:00:00         NaN
 2022-12-31 23:00:00         NaN
 2023-01-01 03:00:00    0.000000
 2023-01-01 07:00:00    0.000000
 2023-01-01 11:00:00    0.000000
                          ...   
 2023-12-30 03:00:00   -0.110724
 2023-12-30 07:00:00   -0.140447
 2023-12-30 11:00:00   -0.153566
 2023-12-30 15:00:00   -0.120777
 2023-12-30 19:00:00   -0.077856
 Freq: 4h, Length: 2185, dtype: float64}